In [8]:
#%% Setting Up
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import xarray as xr
import xesmf as xe
import networkx as nx
import rioxarray as rxr

import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt

from shapely.geometry import Point
from shapely.geometry import Polygon

import glob
import os
import itertools
import tqdm
import gc
import time
import pickle

from joblib import Parallel, delayed

import rioxarray as rxr

import configparser
cfg = configparser.ConfigParser()
cfg.optionxform = str
cfg.read('/home/sarth/rootdir/datadir/assets/defaults.ini')
cfg = {s: dict(cfg.items(s)) for s in cfg.sections()}
PATHS = cfg['PATHS']

print("Setting up...")

Setting up...


In [9]:
#%% Region-Specific: CAMELS-US
DIRNAME = '03min_GloFAS_CAMELS-US'
SAVE_PATH = os.path.join(PATHS['devp_datasets'], DIRNAME)
resolution = 0.05
lon_360_180 = lambda x: (x + 180) % 360 - 180 # convert 0-360 to -180-180
lon_180_360 = lambda x: x % 360 # convert -180-180 to 0-360
region_bounds = {
    'minx': -130,
    'miny': 20,
    'maxx': -65,
    'maxy': 50
}

camels_graph = pd.read_csv(os.path.join(SAVE_PATH, 'nested_gauges', 'graph_attributes_with_nesting.csv'), index_col=0)
camels_graph.index = camels_graph.index.map(lambda x: str(x).zfill(8))
camels_graph['huc_02'] = camels_graph['huc_02'].map(lambda x: str(x).zfill(2))
# camels_graph = camels_graph[camels_graph['nesting'].isin(['not_nested', 'nested_downstream'])]
camels_graph = camels_graph.reset_index()
print(f"Number of catmt's with nesting: {len(camels_graph)}")

Number of catmt's with nesting: 395


In [10]:
def idx_to_map(ds, var_name):
    lats = ds.lat.values
    lons = ds.lon.values
    catmt_var_map = xr.DataArray(
        np.zeros((len(lats), len(lons)), dtype = np.float32)*np.nan,
        dims = ['lat', 'lon'],
        coords = {'lat': lats, 'lon': lons}
    )
    for idx in ds.idx.values:
        lat, lon = ds['idx2lat'].sel(idx = idx).values, ds['idx2lon'].sel(idx = idx).values
        catmt_var_map.loc[dict(lat = lat, lon = lon)] = ds[var_name].sel(idx = idx).values
    return catmt_var_map

START_DATE = pd.Timestamp('1998-01-01')
END_DATE = pd.Timestamp('2022-12-31')

# Helper Function

In [4]:
def process_catmt_final(huc, gauge_id):
    warnings.filterwarnings('ignore')
    
    catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))
    catmt = catmt[sorted(catmt.data_vars)]

    LUMPED_PATH = os.path.join(SAVE_PATH, 'lumped_inventory')

    var_names = [var for var in catmt.data_vars if (var.startswith('dynamic_') and (not 'USGS' in var) and (not 'discharge' in var))]
    var_names.extend(['encoding_solar_insolation'])
    catmt_area_weighted = (catmt[var_names] * catmt['static_GloFAS_cellarea_km2']).sum(dim='idx') / catmt['static_GloFAS_cellarea_km2'].sum(dim='idx')
    catmt_area_weighted = catmt_area_weighted[sorted(catmt_area_weighted.data_vars)]

    var_names = [var for var in catmt.data_vars if (var.startswith('dynamic_') and (not 'USGS' in var) and (not 'discharge' in var))]
    var_names.extend(['encoding_solar_insolation', 'idx2lat', 'idx2lon', 'mask'])
    var_names = [var for var in catmt.data_vars if not var in var_names]

    # For .sel(idx=0)
    catmt_lumped_sel = catmt[var_names].copy()
    catmt_lumped_sel = catmt_lumped_sel[['dynamic_GloFAS_discharge_mm', 'static_uparea', 'encoding_transformed_longitude', 'encoding_transformed_latitude']]
    catmt_lumped_sel = catmt_lumped_sel.sel(idx=0)

    # For .area_weighted
    # Drop ['dynamic_GloFAS_discharge_mm', 'static_uparea'] from var_names
    var_names = [var for var in var_names if not (var in ['dynamic_GloFAS_discharge_mm', 'static_uparea', 'encoding_transformed_longitude', 'encoding_transformed_latitude'])]
    var_names = [var for var in var_names if not var.startswith('static_ERA5_type') or var.startswith('static_ERA5_soil_type')]
    catmt_area_weighted_2 = (catmt[var_names] * catmt['static_GloFAS_cellarea_km2']).sum(dim='idx') / catmt['static_GloFAS_cellarea_km2'].sum(dim='idx')
    catmt_area_weighted_2 = catmt_area_weighted_2[sorted(catmt_area_weighted_2.data_vars)]

    catmt_lumped = xr.merge([
        catmt_area_weighted,
        catmt_lumped_sel,
        catmt_area_weighted_2
    ])

    # Save the area-weighted variables
    os.makedirs(os.path.join(LUMPED_PATH, huc), exist_ok=True)
    catmt_lumped.to_zarr(os.path.join(LUMPED_PATH, huc, f'{gauge_id}.zarr'), mode='w')

    # Clean up
    del catmt, catmt_area_weighted
    gc.collect()

    return None

In [ ]:
# def process_catmt(huc, gauge_id):
#     warnings.filterwarnings('ignore')
    
#     catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))
#     catmt = catmt[sorted(catmt.data_vars)]

#     LUMPED_PATH = os.path.join(SAVE_PATH, 'lumped_inventory')

#     var_names = [var for var in catmt.data_vars if (var.startswith('dynamic_') and (not 'USGS' in var) and (not 'discharge' in var))]
#     var_names.extend(['encoding_solar_insolation'])
#     catmt_area_weighted = (catmt[var_names] * catmt['static_GloFAS_cellarea_km2']).sum(dim='idx') / catmt['static_GloFAS_cellarea_km2'].sum(dim='idx')
#     catmt_area_weighted = catmt_area_weighted[sorted(catmt_area_weighted.data_vars)]

#     # Save the area-weighted variables
#     os.makedirs(os.path.join(LUMPED_PATH, huc), exist_ok=True)
#     catmt_area_weighted.to_zarr(os.path.join(LUMPED_PATH, huc, f'{gauge_id}.zarr'), mode='w')

#     # Clean up
#     del catmt, catmt_area_weighted
#     gc.collect()

#     return None

In [ ]:
# camels_graph = camels_graph.sort_values(by=['area_geospa_fabric'], ascending=True)

In [ ]:
# for idx, row in itertools.islice(tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph)), 0, None, 1):
#     huc, gauge_id = row['huc_02'], row['gauge_id']
#     process_catmt_final(huc, gauge_id)

In [ ]:
# camels_graph.iloc[:8]['gauge_id'].values # Run monthly inventory for these gauges again. Corrupted accidentally.

array(['03140000', '03384450', '03280700', '03078000', '03187500',
       '03021350', '03010655', '03186500'], dtype=object)

In [5]:
with Parallel(n_jobs=16, verbose=10) as parallel:
    parallel(
        delayed(process_catmt_final)(row['huc_02'], row['gauge_id']) for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph))
    )

 12%|█▏        | 48/395 [02:12<18:39,  3.23s/it][Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:  2.3min
[Parallel(n_jobs=16)]: Done  29 tasks      | elapsed:  2.8min
 24%|██▍       | 96/395 [04:57<17:10,  3.45s/it][Parallel(n_jobs=16)]: Done  66 tasks      | elapsed:  5.0min
[Parallel(n_jobs=16)]: Done  81 tasks      | elapsed:  5.8min
100%|██████████| 395/395 [14:20<00:00,  2.18s/it]
[Parallel(n_jobs=16)]: Done 360 tasks      | elapsed: 14.4min
[Parallel(n_jobs=16)]: Done 395 out of 395 | elapsed: 14.6min finished


In [ ]:
# def process_catmt(huc, gauge_id):
#     warnings.filterwarnings('ignore')
    
#     LUMPED_PATH = os.path.join(SAVE_PATH, 'lumped_inventory')

#     catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))
#     catmt = catmt[sorted(catmt.data_vars)]

#     catmt_lumped = xr.open_zarr(os.path.join(LUMPED_PATH, huc, f'{gauge_id}.zarr'))

#     var_names = [var for var in catmt.data_vars if (var.startswith('dynamic_') and (not 'USGS' in var) and (not 'discharge' in var))]
#     var_names.extend(['encoding_solar_insolation', 'idx2lat', 'idx2lon', 'mask'])
#     var_names = [var for var in catmt.data_vars if not var in var_names]

#     # For .sel(idx=0)
#     catmt_lumped_sel = catmt[var_names].copy()
#     catmt_lumped_sel = catmt_lumped_sel[['dynamic_GloFAS_discharge_mm', 'static_uparea', 'encoding_transformed_longitude', 'encoding_transformed_latitude']]
#     catmt_lumped_sel = catmt_lumped_sel.sel(idx=0)

#     # For .area_weighted
#     # Drop ['dynamic_GloFAS_discharge_mm', 'static_uparea'] from var_names
#     var_names = [var for var in var_names if not (var in ['dynamic_GloFAS_discharge_mm', 'static_uparea'])]
#     var_names = [var for var in var_names if not var.startswith('static_ERA5_type') or var.startswith('static_ERA5_soil_type')]
#     catmt_area_weighted = (catmt[var_names] * catmt['static_GloFAS_cellarea_km2']).sum(dim='idx') / catmt['static_GloFAS_cellarea_km2'].sum(dim='idx')
#     catmt_area_weighted = catmt_area_weighted[sorted(catmt_area_weighted.data_vars)]

#     catmt_lumped = xr.open_zarr(os.path.join(SAVE_PATH, 'lumped_inventory', huc, f'{gauge_id}.zarr'))
#     catmt_lumped = xr.merge([catmt_lumped, catmt_lumped_sel, catmt_area_weighted])
#     catmt_lumped[sorted(catmt_lumped.data_vars)]

#     # Replace the value for 'encoding_transformed_longitude', 'encoding_transformed_latitude'
#     # catmt_lumped['encoding_transformed_longitude'] = catmt['encoding_transformed_longitude'].sel(idx=0)
#     # catmt_lumped['encoding_transformed_latitude'] = catmt['encoding_transformed_latitude'].sel(idx=0)

#     # Save the area-weighted variables
#     os.makedirs(os.path.join(LUMPED_PATH, huc), exist_ok=True)
#     catmt_area_weighted.to_zarr(os.path.join(LUMPED_PATH, huc, f'{gauge_id}.zarr'), mode='a')

#     # Clean up
#     del catmt, catmt_area_weighted
#     gc.collect()

#     return None

In [ ]:
# with Parallel(n_jobs=8, verbose=10) as parallel:
#     parallel(
#         delayed(process_catmt)(row['huc_02'], row['gauge_id']) for idx, row in tqdm.tqdm(camels_graph.iterrows(), total=len(camels_graph))
#     )

In [7]:
row = camels_graph.iloc[0]
huc, gauge_id = row['huc_02'], row['gauge_id']
catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'lumped_inventory', huc, f'{gauge_id}.zarr'))
data_vars = list(sorted(catmt.data_vars))
# print(f"Length of idx vars: {len(catmt.idx.values)}")
catmt.close()
print(f"Length of data_vars: {len(data_vars)}")
data_vars

Length of data_vars: 147


['dynamic_Daymet_dayl',
 'dynamic_Daymet_prcp',
 'dynamic_Daymet_srad',
 'dynamic_Daymet_swe',
 'dynamic_Daymet_tmax',
 'dynamic_Daymet_tmin',
 'dynamic_Daymet_vp',
 'dynamic_ERA5-Land_dewpoint_temperature_2m_max',
 'dynamic_ERA5-Land_dewpoint_temperature_2m_min',
 'dynamic_ERA5-Land_leaf_area_index_high_vegetation',
 'dynamic_ERA5-Land_leaf_area_index_low_vegetation',
 'dynamic_ERA5-Land_potential_evaporation_sum',
 'dynamic_ERA5-Land_runoff_sum',
 'dynamic_ERA5-Land_snow_cover',
 'dynamic_ERA5-Land_snow_depth',
 'dynamic_ERA5-Land_snowfall_sum',
 'dynamic_ERA5-Land_snowmelt_sum',
 'dynamic_ERA5-Land_sub_surface_runoff_sum',
 'dynamic_ERA5-Land_surface_net_solar_radiation_sum',
 'dynamic_ERA5-Land_surface_net_thermal_radiation_sum',
 'dynamic_ERA5-Land_surface_pressure',
 'dynamic_ERA5-Land_surface_runoff_sum',
 'dynamic_ERA5-Land_temperature_2m_max',
 'dynamic_ERA5-Land_temperature_2m_min',
 'dynamic_ERA5-Land_total_evaporation_sum',
 'dynamic_ERA5-Land_total_precipitation_sum',
 'dy

# Development

In [59]:
camels_graph.shape

(395, 13)

In [54]:
camels_graph = camels_graph[camels_graph['huc_02'] == '05']

In [55]:
row = camels_graph.iloc[0]
huc, gauge_id = row['huc_02'], row['gauge_id']
print(f"Processing HUC: {huc}, Gauge ID: {gauge_id}")

Processing HUC: 05, Gauge ID: 03164000


In [56]:
catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'lumped_inventory', huc, f'{gauge_id}.zarr'))
catmt = catmt[sorted(catmt.data_vars)]
catmt

<xarray.Dataset> Size: 4MB
Dimensions:                                              (time: 9125, lead: 10)
Coordinates:
  * time                                                 (time) datetime64[ns] 73kB ...
  * lead                                                 (lead) int32 40B 24 ...
Data variables: (12/145)
    dynamic_Daymet_dayl                                  (time) float32 36kB dask.array<chunksize=(2282,), meta=np.ndarray>
    dynamic_Daymet_prcp                                  (time) float32 36kB dask.array<chunksize=(2282,), meta=np.ndarray>
    dynamic_Daymet_srad                                  (time) float32 36kB dask.array<chunksize=(2282,), meta=np.ndarray>
    dynamic_Daymet_swe                                   (time) float32 36kB dask.array<chunksize=(2282,), meta=np.ndarray>
    dynamic_Daymet_tmax                                  (time) float32 36kB dask.array<chunksize=(2282,), meta=np.ndarray>
    dynamic_Daymet_tmin                                  (time) float32 36kB dask.array<chunksize=(2282,), meta=np.ndarray>
    ...                                                   ...
    static_terrain_upa_50%                               float32 4B ...
    static_terrain_upa_mean                              float32 4B ...
    static_terrain_upa_std                               float32 4B ...
    static_terrain_wth_50%                               float32 4B ...
    static_terrain_wth_mean                              float32 4B ...
    static_terrain_wth_std                               float32 4B ...

In [27]:
catmt = xr.open_zarr(os.path.join(SAVE_PATH, 'inventory', huc, f'{gauge_id}.zarr'))
catmt = catmt[sorted(catmt.data_vars)]
catmt

<xarray.Dataset> Size: 13MB
Dimensions:                                              (time: 9125, idx: 3,
                                                          lead: 10, lat: 3,
                                                          lon: 2)
Coordinates:
  * time                                                 (time) datetime64[ns] 73kB ...
  * lead                                                 (lead) int32 40B 24 ...
  * lon                                                  (lon) float32 8B -81...
  * lat                                                  (lat) float32 12B 40...
  * idx                                                  (idx) int32 12B 0 1 2
Data variables: (12/152)
    dynamic_Daymet_dayl                                  (time, idx) float32 110kB dask.array<chunksize=(9125, 3), meta=np.ndarray>
    dynamic_Daymet_prcp                                  (time, idx) float32 110kB dask.array<chunksize=(9125, 3), meta=np.ndarray>
    dynamic_Daymet_srad                                  (time, idx) float32 110kB dask.array<chunksize=(9125, 3), meta=np.ndarray>
    dynamic_Daymet_swe                                   (time, idx) float32 110kB dask.array<chunksize=(9125, 3), meta=np.ndarray>
    dynamic_Daymet_tmax                                  (time, idx) float32 110kB dask.array<chunksize=(9125, 3), meta=np.ndarray>
    dynamic_Daymet_tmin                                  (time, idx) float32 110kB dask.array<chunksize=(9125, 3), meta=np.ndarray>
    ...                                                   ...
    static_terrain_upa_mean                              (idx) float32 12B dask.array<chunksize=(3,), meta=np.ndarray>
    static_terrain_upa_std                               (idx) float32 12B dask.array<chunksize=(3,), meta=np.ndarray>
    static_terrain_wth_50%                               (idx) float32 12B dask.array<chunksize=(3,), meta=np.ndarray>
    static_terrain_wth_mean                              (idx) float32 12B dask.array<chunksize=(3,), meta=np.ndarray>
    static_terrain_wth_std                               (idx) float32 12B dask.array<chunksize=(3,), meta=np.ndarray>
    static_uparea                                        (idx) float32 12B dask.array<chunksize=(3,), meta=np.ndarray>
Attributes:
    watershed:            CAMELS-US/HUC:05/GaugeID:03140000
    gauge_idx:            [0]
    resolution(degrees):  0.05
    resolution(arcmins):  3
    edges_idx:            [[2, 1], [1, 0]]
    drainage_area_km2:    71.31

In [ ]:
# USGS, GloFAS_discharge, encoding (excluding solar insolation),
# Mean: ClimSumm, static_GloFAS/terrain/HWSD
# sel idx 0: static_uparea

In [45]:
var_names = [var for var in catmt.data_vars if (var.startswith('dynamic_') and (not 'USGS' in var) and (not 'discharge' in var))]
var_names.extend(['encoding_solar_insolation', 'idx2lat', 'idx2lon', 'mask'])
var_names = [var for var in catmt.data_vars if not var in var_names]

# For .sel(idx=0)
catmt_lumped = catmt[var_names].copy()
catmt_lumped_sel = catmt_lumped[['dynamic_GloFAS_discharge_mm', 'static_uparea']]
catmt_lumped_sel = catmt_lumped_sel.sel(idx=0)

# For .area_weighted
# Drop ['dynamic_GloFAS_discharge_mm', 'static_uparea'] from var_names
var_names = [var for var in var_names if not (var in ['dynamic_GloFAS_discharge_mm', 'static_uparea'])]
var_names = [var for var in var_names if not var.startswith('static_ERA5_type') or var.startswith('static_ERA5_soil_type')]
catmt_area_weighted = (catmt[var_names] * catmt['static_GloFAS_cellarea_km2']).sum(dim='idx') / catmt['static_GloFAS_cellarea_km2'].sum(dim='idx')
catmt_area_weighted = catmt_area_weighted[sorted(catmt_area_weighted.data_vars)]

catmt_lumped = xr.open_zarr(os.path.join(SAVE_PATH, 'lumped_inventory', huc, f'{gauge_id}.zarr'))
catmt_lumped = xr.merge([catmt_lumped, catmt_lumped_sel, catmt_area_weighted])
catmt_lumped[sorted(catmt_lumped.data_vars)]

<xarray.Dataset> Size: 4MB
Dimensions:                                              (time: 9125, lead: 10)
Coordinates:
  * lead                                                 (lead) int32 40B 24 ...
  * time                                                 (time) datetime64[ns] 73kB ...
    idx                                                  int32 4B 0
Data variables: (12/147)
    dynamic_Daymet_dayl                                  (time) float32 36kB dask.array<chunksize=(9125,), meta=np.ndarray>
    dynamic_Daymet_prcp                                  (time) float32 36kB dask.array<chunksize=(9125,), meta=np.ndarray>
    dynamic_Daymet_srad                                  (time) float32 36kB dask.array<chunksize=(9125,), meta=np.ndarray>
    dynamic_Daymet_swe                                   (time) float32 36kB dask.array<chunksize=(9125,), meta=np.ndarray>
    dynamic_Daymet_tmax                                  (time) float32 36kB dask.array<chunksize=(9125,), meta=np.ndarray>
    dynamic_Daymet_tmin                                  (time) float32 36kB dask.array<chunksize=(9125,), meta=np.ndarray>
    ...                                                   ...
    static_terrain_upa_mean                              float32 4B dask.array<chunksize=(), meta=np.ndarray>
    static_terrain_upa_std                               float32 4B dask.array<chunksize=(), meta=np.ndarray>
    static_terrain_wth_50%                               float32 4B dask.array<chunksize=(), meta=np.ndarray>
    static_terrain_wth_mean                              float32 4B dask.array<chunksize=(), meta=np.ndarray>
    static_terrain_wth_std                               float32 4B dask.array<chunksize=(), meta=np.ndarray>
    static_uparea                                        float32 4B dask.array<chunksize=(), meta=np.ndarray>

In [42]:
catmt_lumped_sel

<xarray.Dataset> Size: 110kB
Dimensions:                      (time: 9125)
Coordinates:
  * time                         (time) datetime64[ns] 73kB 1998-01-01 ... 20...
    idx                          int32 4B 0
Data variables:
    dynamic_GloFAS_discharge_mm  (time) float32 36kB dask.array<chunksize=(9125,), meta=np.ndarray>
    static_uparea                float32 4B dask.array<chunksize=(), meta=np.ndarray>
Attributes:
    watershed:            CAMELS-US/HUC:05/GaugeID:03140000
    gauge_idx:            [0]
    resolution(degrees):  0.05
    resolution(arcmins):  3
    edges_idx:            [[2, 1], [1, 0]]
    drainage_area_km2:    71.31

In [43]:
catmt_area_weighted

<xarray.Dataset> Size: 219kB
Dimensions:                                 (time: 9125)
Coordinates:
  * time                                    (time) datetime64[ns] 73kB 1998-0...
Data variables: (12/78)
    encoding_sine_dayofyear                 (time) float32 36kB dask.array<chunksize=(9125,), meta=np.ndarray>
    encoding_sine_month                     (time) float32 36kB dask.array<chunksize=(9125,), meta=np.ndarray>
    encoding_sine_weekofyear                (time) float32 36kB dask.array<chunksize=(9125,), meta=np.ndarray>
    encoding_transformed_latitude           float32 4B dask.array<chunksize=(), meta=np.ndarray>
    encoding_transformed_longitude          float32 4B dask.array<chunksize=(), meta=np.ndarray>
    outlet_USGS_Q_mm                        (time) float32 36kB dask.array<chunksize=(9125,), meta=np.ndarray>
    ...                                      ...
    static_terrain_upa_50%                  float32 4B dask.array<chunksize=(), meta=np.ndarray>
    static_terrain_upa_mean                 float32 4B dask.array<chunksize=(), meta=np.ndarray>
    static_terrain_upa_std                  float32 4B dask.array<chunksize=(), meta=np.ndarray>
    static_terrain_wth_50%                  float32 4B dask.array<chunksize=(), meta=np.ndarray>
    static_terrain_wth_mean                 float32 4B dask.array<chunksize=(), meta=np.ndarray>
    static_terrain_wth_std                  float32 4B dask.array<chunksize=(), meta=np.ndarray>

In [31]:
catmt[sorted(var_names)]

<xarray.Dataset> Size: 330kB
Dimensions:                                 (time: 9125, idx: 3)
Coordinates:
  * time                                    (time) datetime64[ns] 73kB 1998-0...
  * idx                                     (idx) int32 12B 0 1 2
Data variables: (12/82)
    dynamic_GloFAS_discharge_mm             (time, idx) float32 110kB dask.array<chunksize=(9125, 3), meta=np.ndarray>
    encoding_sine_dayofyear                 (time) float32 36kB dask.array<chunksize=(9125,), meta=np.ndarray>
    encoding_sine_month                     (time) float32 36kB dask.array<chunksize=(9125,), meta=np.ndarray>
    encoding_sine_weekofyear                (time) float32 36kB dask.array<chunksize=(9125,), meta=np.ndarray>
    encoding_transformed_latitude           (idx) float32 12B dask.array<chunksize=(3,), meta=np.ndarray>
    encoding_transformed_longitude          (idx) float32 12B dask.array<chunksize=(3,), meta=np.ndarray>
    ...                                      ...
    static_terrain_upa_mean                 (idx) float32 12B dask.array<chunksize=(3,), meta=np.ndarray>
    static_terrain_upa_std                  (idx) float32 12B dask.array<chunksize=(3,), meta=np.ndarray>
    static_terrain_wth_50%                  (idx) float32 12B dask.array<chunksize=(3,), meta=np.ndarray>
    static_terrain_wth_mean                 (idx) float32 12B dask.array<chunksize=(3,), meta=np.ndarray>
    static_terrain_wth_std                  (idx) float32 12B dask.array<chunksize=(3,), meta=np.ndarray>
    static_uparea                           (idx) float32 12B dask.array<chunksize=(3,), meta=np.ndarray>
Attributes:
    watershed:            CAMELS-US/HUC:05/GaugeID:03140000
    gauge_idx:            [0]
    resolution(degrees):  0.05
    resolution(arcmins):  3
    edges_idx:            [[2, 1], [1, 0]]
    drainage_area_km2:    71.31

In [ ]:

catmt_area_weighted = (catmt[var_names] * catmt['static_GloFAS_cellarea_km2']).sum(dim='idx') / catmt['static_GloFAS_cellarea_km2'].sum(dim='idx')
catmt_area_weighted = catmt_area_weighted[sorted(catmt_area_weighted.data_vars)]
catmt_area_weighted

<xarray.Dataset> Size: 4MB
Dimensions:                                              (time: 9125, lead: 10)
Coordinates:
  * time                                                 (time) datetime64[ns] 73kB ...
  * lead                                                 (lead) int32 40B 24 ...
Data variables: (12/67)
    dynamic_Daymet_dayl                                  (time) float32 36kB dask.array<chunksize=(2282,), meta=np.ndarray>
    dynamic_Daymet_prcp                                  (time) float32 36kB dask.array<chunksize=(2282,), meta=np.ndarray>
    dynamic_Daymet_srad                                  (time) float32 36kB dask.array<chunksize=(2282,), meta=np.ndarray>
    dynamic_Daymet_swe                                   (time) float32 36kB dask.array<chunksize=(2282,), meta=np.ndarray>
    dynamic_Daymet_tmax                                  (time) float32 36kB dask.array<chunksize=(2282,), meta=np.ndarray>
    dynamic_Daymet_tmin                                  (time) float32 36kB dask.array<chunksize=(2282,), meta=np.ndarray>
    ...                                                   ...
    dynamic_HRES_10m_u_component_of_wind                 (time, lead) float32 365kB dask.array<chunksize=(2282, 3), meta=np.ndarray>
    dynamic_HRES_10m_v_component_of_wind                 (time, lead) float32 365kB dask.array<chunksize=(2282, 3), meta=np.ndarray>
    dynamic_HRES_2m_temperature                          (time, lead) float32 365kB dask.array<chunksize=(2282, 3), meta=np.ndarray>
    dynamic_HRES_surface_pressure                        (time, lead) float32 365kB dask.array<chunksize=(2282, 3), meta=np.ndarray>
    dynamic_HRES_total_precipitation_24hr                (time, lead) float32 365kB dask.array<chunksize=(2282, 3), meta=np.ndarray>
    encoding_solar_insolation                            (time) float32 36kB dask.array<chunksize=(2282,), meta=np.ndarray>

In [10]:
catmt

<xarray.Dataset> Size: 512MB
Dimensions:                                              (time: 9125, idx: 124,
                                                          lead: 10, lat: 14,
                                                          lon: 17)
Coordinates:
  * lat                                                  (lat) float32 56B 36...
  * time                                                 (time) datetime64[ns] 73kB ...
  * lead                                                 (lead) int32 40B 24 ...
  * lon                                                  (lon) float32 68B -8...
  * idx                                                  (idx) int32 496B 0 ....
Data variables: (12/152)
    dynamic_Daymet_dayl                                  (time, idx) float32 5MB dask.array<chunksize=(2282, 62), meta=np.ndarray>
    dynamic_Daymet_prcp                                  (time, idx) float32 5MB dask.array<chunksize=(2282, 62), meta=np.ndarray>
    dynamic_Daymet_srad                                  (time, idx) float32 5MB dask.array<chunksize=(2282, 62), meta=np.ndarray>
    dynamic_Daymet_swe                                   (time, idx) float32 5MB dask.array<chunksize=(2282, 62), meta=np.ndarray>
    dynamic_Daymet_tmax                                  (time, idx) float32 5MB dask.array<chunksize=(2282, 62), meta=np.ndarray>
    dynamic_Daymet_tmin                                  (time, idx) float32 5MB dask.array<chunksize=(2282, 62), meta=np.ndarray>
    ...                                                   ...
    static_terrain_upa_mean                              (idx) float32 496B dask.array<chunksize=(124,), meta=np.ndarray>
    static_terrain_upa_std                               (idx) float32 496B dask.array<chunksize=(124,), meta=np.ndarray>
    static_terrain_wth_50%                               (idx) float32 496B dask.array<chunksize=(124,), meta=np.ndarray>
    static_terrain_wth_mean                              (idx) float32 496B dask.array<chunksize=(124,), meta=np.ndarray>
    static_terrain_wth_std                               (idx) float32 496B dask.array<chunksize=(124,), meta=np.ndarray>
    static_uparea                                        (idx) float32 496B dask.array<chunksize=(124,), meta=np.ndarray>
Attributes:
    watershed:            CAMELS-US/HUC:05/GaugeID:03164000
    gauge_idx:            [0]
    resolution(degrees):  0.05
    resolution(arcmins):  3
    edges_idx:            [[123, 122], [122, 120], [121, 120], [120, 116], [1...
    drainage_area_km2:    2963.7

In [12]:
row = camels_graph.iloc[0]
temp1 = xr.open_zarr(os.path.join(SAVE_PATH, 'lumped_inventory', row['huc_02'], f"{row['gauge_id']}.zarr"))
temp1 = temp1['dynamic_GloFAS_discharge_mm']
temp1

<xarray.DataArray 'dynamic_GloFAS_discharge_mm' (time: 9125)> Size: 36kB
dask.array<open_dataset-dynamic_GloFAS_discharge_mm, shape=(9125,), dtype=float32, chunksize=(1141,), chunktype=numpy.ndarray>
Coordinates:
    idx      int32 4B ...
  * time     (time) datetime64[ns] 73kB 1998-01-01 1998-01-02 ... 2022-12-31

In [13]:
row = camels_graph.iloc[1]
temp2 = xr.open_zarr(os.path.join(SAVE_PATH, 'lumped_inventory', row['huc_02'], f"{row['gauge_id']}.zarr"))
temp2 = temp2['dynamic_GloFAS_discharge_mm']
temp2

<xarray.DataArray 'dynamic_GloFAS_discharge_mm' (time: 9125)> Size: 36kB
dask.array<open_dataset-dynamic_GloFAS_discharge_mm, shape=(9125,), dtype=float32, chunksize=(1141,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 73kB 1998-01-01 1998-01-02 ... 2022-12-31
    idx      int32 4B ...

In [14]:
temp1.values

array([0.00465842, 0.00445138, 0.0042961 , ...,        nan,        nan,
              nan], shape=(9125,), dtype=float32)

In [15]:
temp2.values

array([0.22230518, 0.3635895 , 0.53806657, ...,        nan,        nan,
              nan], shape=(9125,), dtype=float32)

In [16]:
temp1.values == temp2.values

array([False, False, False, ..., False, False, False], shape=(9125,))